# FX Trading Strategy Using Markov Chains
This Jupyter notebook demonstrates a simple financial model using Markov Chains to identify profitable trading opportunities in the Forex (FX) market. 
The notebook includes the following steps:

- Load Historical FX Data: We'll use the yfinance library to load historical data.
- Define and Train the Markov Chain Model: We'll use PyTorch to build and train the model.
- Backtest the Model: We'll use a different time frame to test the model.
- Evaluate Performance: We'll evaluate the model's performance over the trading simulation.

In [ ]:
# Install necessary libraries
%pip install yfinance pandas
%pip install torch

# Import libraries
import yfinance as yf
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load Historical FX Data
def load_fx_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data['Return'] = data['Close'].pct_change()
    data.dropna(inplace=True)
    return data

In [ ]:
# Example: Load EUR/USD data
fx_data = load_fx_data('EURUSD=X', '2015-01-01', '2019-12-31')
fx_data.head()

In [ ]:
# Plot the closing prices
plt.figure(figsize=(14, 7))
plt.plot(fx_data.index, fx_data['Close'], label='Close Price')
plt.title('EUR/USD Closing Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Plot the daily returns
plt.figure(figsize=(14, 7))
plt.plot(fx_data.index, fx_data['Return'], label='Daily Return')
plt.title('EUR/USD Daily Returns')
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.legend()
plt.show()

In [ ]:
# Define the Markov Chain Model
class MarkovChainModel:
    def __init__(self, states):
        self.states = states
        self.num_states = len(states)
        self.transition_matrix = torch.zeros((self.num_states, self.num_states))
    
    def fit(self, data):
        for i in range(len(data) - 1):
            current_state = self.states.index(self.get_state(data[i]))
            next_state = self.states.index(self.get_state(data[i + 1]))
            self.transition_matrix[current_state][next_state] += 1
        self.transition_matrix = self.transition_matrix / self.transition_matrix.sum(axis=1, keepdim=True)
    
    def get_state(self, return_value):
        if return_value > 0.001:
            return 'up'
        elif return_value < -0.001:
            return 'down'
        else:
            return 'stable'
    
    def predict(self, current_state):
        current_state_idx = self.states.index(current_state)
        next_state_idx = torch.multinomial(self.transition_matrix[current_state_idx], 1).item()
        return self.states[next_state_idx]

In [ ]:
# Train the Model
states = ['up', 'down', 'stable']
model = MarkovChainModel(states)
model.fit(fx_data['Return'].values)

In [ ]:
# Backtest the Model
def backtest_model(model, data, initial_balance=10000):
    balance = initial_balance
    position = 0  # 0 means no position, 1 means long, -1 means short
    trade_log = []

    balances = [balance]
    for i in range(1, len(data)):
        current_state = model.get_state(data['Return'].values[i - 1])
        predicted_state = model.predict(current_state)
        
        # Implement simple trading strategy
        if predicted_state == 'up' and position <= 0:
            position = 1
            trade_log.append((data.index[i], 'Buy', data['Close'].values[i]))
        elif predicted_state == 'down' and position >= 0:
            position = -1
            trade_log.append((data.index[i], 'Sell', data['Close'].values[i]))
        elif predicted_state == 'stable':
            if position == 1:
                trade_log.append((data.index[i], 'Sell', data['Close'].values[i]))
                position = 0
            elif position == -1:
                trade_log.append((data.index[i], 'Buy', data['Close'].values[i]))
                position = 0
        
        balance += position * (data['Close'].values[i] - data['Close'].values[i - 1])
        balances.append(balance)
    
    return trade_log, balances

In [ ]:
# Backtest the model with a different time frame
backtest_data = load_fx_data('EURUSD=X', '2020-01-01', '2023-01-01')
trade_log, balances = backtest_model(model, backtest_data)

In [ ]:
# Ensure the lengths match
dates = backtest_data.index[1:]

# Plot the balance over time
plt.figure(figsize=(14, 7))
plt.plot(dates, balances[:-1], label='Balance')
plt.title('Backtesting Balance Over Time')
plt.xlabel('Date')
plt.ylabel('Balance')
plt.legend()
plt.show()

In [ ]:
# Plot the closing prices with buy and sell signals
plt.figure(figsize=(14, 7))
plt.plot(backtest_data.index, backtest_data['Close'], label='Close Price')
buy_signals = [log[0] for log in trade_log if log[1] == 'Buy']
sell_signals = [log[0] for log in trade_log if log[1] == 'Sell']
plt.scatter(buy_signals, backtest_data.loc[buy_signals]['Close'], marker='^', color='g', label='Buy Signal', s=100)
plt.scatter(sell_signals, backtest_data.loc[sell_signals]['Close'], marker='v', color='r', label='Sell Signal', s=100)
plt.title('EUR/USD Closing Prices with Buy and Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Plot the balance over time
def plot_performance(trade_log, initial_balance=10000):
    balance = initial_balance
    balances = [balance]
    dates = [trade_log[0][0]]
    
    for i in range(1, len(trade_log)):
        if trade_log[i][1] == 'Buy':
            balance -= trade_log[i][2]
        else:
            balance += trade_log[i][2]
        balances.append(balance)
        dates.append(trade_log[i][0])
    
    plt.figure(figsize=(12, 6))
    plt.plot(dates, balances, label='Balance')
    plt.xlabel('Date')
    plt.ylabel('Balance')
    plt.title('Trading Performance')
    plt.legend()
    plt.show()

plot_performance(trade_log)